In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from genetic_selection import GeneticSelectionCV
import pandas as pd
from sklearn import metrics
from sklearn import svm
import matplotlib.pyplot as plt

import seaborn as sns